In [112]:
import pandas as pd
import numpy as np
import calendar
from time import time
from datetime import datetime
import re
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score, cross_validate, RandomizedSearchCV
from sklearn.metrics import log_loss, accuracy_score, classification_report
from mlxtend.classifier import StackingClassifier
import pickle
from sklearn.externals import joblib
import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\Xiuquan\Anaconda3\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
data_train = pd.read_csv("train.csv").replace("MENS WEAR","MENSWEAR")

In [3]:
dept_list = sorted(list(data_train.DepartmentDescription.dropna().unique()))

In [4]:
weekdays = list(calendar.day_name)
dept_list_sum = dict.fromkeys(dept_list, np.sum)
weekday_dict = dict.fromkeys(weekdays, np.max)
feature_dict = {"TripType": np.max, 'NumItems': np.sum, 'Return': np.max}
feature_dict = {**feature_dict, **weekday_dict, **dept_list_sum}

In [5]:
def transform_data(data):
    dummies = pd.get_dummies(data.Weekday)
    data[dummies.columns] = dummies
    
    dummies = pd.get_dummies(data.DepartmentDescription)
    dummies = dummies.apply(lambda x: x*data["ScanCount"])
    data[dummies.columns] = dummies 

    data.loc[data.ScanCount < 0, 'Return'] = 1
    data.loc[data.Return != 1, 'Return'] = 0
    
    data = data.rename(columns={"ScanCount":"NumItems"})
    
    grouped = data.groupby("VisitNumber")
    grouped = grouped.aggregate(feature_dict)
    data = grouped[["TripType", "NumItems", "Return"] + weekdays + dept_list]

    return data

In [6]:
data_new = transform_data(data_train)

In [7]:
def add_category_counts(data):
    alist = []
    for array in np.asarray(data.loc[:, dept_list[0]:]):
        count = 0
        count = sum(x > 0 for x in array)
        alist.append(count)
    cat_counts = pd.DataFrame(alist)
    cat_counts = cat_counts.rename(columns={0:"CategoryCount"})
    cat_counts = cat_counts.set_index(data.index)
    data.insert(3, 'CategoryCounts', cat_counts)
    return data

In [8]:
data_new = add_category_counts(data_new)

In [9]:
data_new.shape

(95674, 78)

In [10]:
def fineline_dummies(data):
    values = data.FinelineNumber
    counts = values.value_counts()
    mask = values.isin(counts[counts > 500].index)
    values[~mask] = "-"
    dummies = pd.get_dummies(values).drop('-', axis=1)

    dummies.columns = ['fln_'+str(col) for col in dummies.columns]
    dummies = pd.concat([dummies, data.VisitNumber], axis=1)
    dummies = dummies.groupby("VisitNumber")
    dummies = dummies.aggregate(np.sum)
    return dummies

In [11]:
start = datetime.now()
fln_dummies = fineline_dummies(data_train)
end = datetime.now()
str(end-start)

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


'0:00:11.676816'

In [12]:
data_new = data_new.join(fln_dummies)

In [13]:
data_new.shape

(95674, 351)

In [14]:
def Upc_dummies(data):
    values = data.Upc
    counts = values.value_counts()
    mask = values.isin(counts[counts > 300].index)
    values[~mask] = "-"
    dummies = pd.get_dummies(values).drop('-', axis=1)

    dummies.columns = ['upc_'+str(col) for col in dummies.columns]
    dummies = pd.concat([dummies, data.VisitNumber], axis=1)
    dummies = dummies.groupby("VisitNumber")
    dummies = dummies.aggregate(np.sum)
    return dummies

In [15]:
start = datetime.now()
upc_dummies = Upc_dummies(data_train)
end = datetime.now()
str(end-start)

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


'0:00:04.785053'

In [16]:
data_new = data_new.join(upc_dummies)

In [17]:
data_new.shape

(95674, 451)

In [18]:
X = data_new.drop('TripType', axis=1)

trip_types = sorted(data_train.TripType.unique())
trip_types_map = dict(zip(trip_types, np.arange(0, len(trip_types))))
y = data_new.TripType.map(trip_types_map)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, stratify=y, random_state=42)

In [19]:
len(X.columns)

450

In [20]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Input
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils.np_utils import to_categorical
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.


In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from mlxtend.classifier import StackingClassifier

In [22]:
def create_model_1hl(nodes_l1=512, dropout_l1=0.3):
    model = Sequential()
    model.add(Dense(output_dim=nodes_l1, activation="relu", input_dim=len(X.columns)))
    model.add(Dropout(dropout_l1))
            
    model.add(Dense(output_dim=len(trip_types), activation="softmax"))
    
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    
    return model

In [23]:
clf1 = KerasClassifier(build_fn=create_model_1hl, epochs=30, batch_size=3000, verbose=2)

In [24]:
def create_model_2hl(nodes_l1=400, nodes_l2=120, dropout_l1=0.3, dropout_l2=0.2):
    model = Sequential()
    model.add(Dense(output_dim=nodes_l1, activation="relu", input_dim=len(X.columns)))
    model.add(Dropout(dropout_l1))
    model.add(Dense(output_dim=nodes_l2, activation="relu"))
    model.add(Dropout(dropout_l2))
            
    model.add(Dense(output_dim=len(trip_types), activation="softmax"))
    
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    
    return model

In [25]:
clf2 = KerasClassifier(build_fn=create_model_2hl, epochs=15, batch_size=1000, verbose=2)

In [26]:
def create_model_3hl(nodes_l1=500, nodes_l2=200, nodes_l3=80, dropout_l1=0.3, dropout_l2=0.2, dropout_l3=0.2):
    model = Sequential()
    model.add(Dense(output_dim=nodes_l1, activation="relu", input_dim=len(X.columns)))
    model.add(Dropout(dropout_l1))
    model.add(Dense(output_dim=nodes_l2, activation="relu"))
    model.add(Dropout(dropout_l2))
    model.add(Dense(output_dim=nodes_l2, activation="relu"))
    model.add(Dropout(dropout_l3))
            
    model.add(Dense(output_dim=len(trip_types), activation="softmax"))
    
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    
    return model

In [27]:
clf3 = KerasClassifier(build_fn=create_model_3hl, epochs=15, batch_size=1000, verbose=2)

In [28]:
def create_model_4hl(nodes_l1=512, nodes_l2=256, nodes_l3=128, nodes_l4=64, dropout_l1=0.3, dropout_l2=0.3, dropout_l3=0.2, dropout_l4=0.2):
    model = Sequential()
    model.add(Dense(output_dim=nodes_l1, activation="relu", input_dim=len(X.columns)))
    model.add(Dropout(dropout_l1))
    model.add(Dense(output_dim=nodes_l2, activation="relu"))
    model.add(Dropout(dropout_l2))
    model.add(Dense(output_dim=nodes_l3, activation="relu"))
    model.add(Dropout(dropout_l3))
    model.add(Dense(output_dim=nodes_l4, activation="relu"))
    model.add(Dropout(dropout_l3))
            
    model.add(Dense(output_dim=len(trip_types), activation="softmax"))
    
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    
    return model

In [29]:
clf4 = KerasClassifier(build_fn=create_model_4hl, epochs=15, batch_size=1000, verbose=2)

In [30]:
lr = LogisticRegression(C=0.4, solver='lbfgs', multi_class='multinomial', random_state=42)

In [31]:
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3, clf4], use_probas=True, average_probas=False, meta_classifier=lr)

In [32]:
start = datetime.now()
hist_sclf = sclf.fit(X_train, y_train)
end = datetime.now()
str(end-start)

W0911 14:26:37.975053  6556 deprecation_wrapper.py:119] From C:\Users\Xiuquan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:66: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=450, units=512)`
  This is separate from the ipykernel package so we can avoid doing imports until
W0911 14:26:38.006036  6556 deprecation_wrapper.py:119] From C:\Users\Xiuquan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:541: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0911 14:26:38.010036  6556 deprecation_wrapper.py:119] From C:\Users\Xiuquan\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4432: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0911 14:26:38.038013  6556 depre

Epoch 1/30
 - 5s - loss: 2.6167 - acc: 0.4291
Epoch 2/30
 - 5s - loss: 1.7164 - acc: 0.6006
Epoch 3/30
 - 4s - loss: 1.3510 - acc: 0.6418
Epoch 4/30
 - 5s - loss: 1.1710 - acc: 0.6635
Epoch 5/30
 - 5s - loss: 1.0680 - acc: 0.6831
Epoch 6/30
 - 5s - loss: 1.0094 - acc: 0.6911
Epoch 7/30
 - 4s - loss: 0.9602 - acc: 0.6994
Epoch 8/30
 - 5s - loss: 0.9254 - acc: 0.7062
Epoch 9/30
 - 4s - loss: 0.8946 - acc: 0.7137
Epoch 10/30
 - 5s - loss: 0.8702 - acc: 0.7160
Epoch 11/30
 - 5s - loss: 0.8453 - acc: 0.7245
Epoch 12/30
 - 5s - loss: 0.8288 - acc: 0.7266
Epoch 13/30
 - 4s - loss: 0.8089 - acc: 0.7319
Epoch 14/30
 - 4s - loss: 0.7952 - acc: 0.7343
Epoch 15/30
 - 5s - loss: 0.7797 - acc: 0.7383
Epoch 16/30
 - 4s - loss: 0.7676 - acc: 0.7412
Epoch 17/30
 - 5s - loss: 0.7551 - acc: 0.7447
Epoch 18/30
 - 6s - loss: 0.7422 - acc: 0.7483
Epoch 19/30
 - 5s - loss: 0.7330 - acc: 0.7513
Epoch 20/30
 - 5s - loss: 0.7211 - acc: 0.7542
Epoch 21/30
 - 5s - loss: 0.7130 - acc: 0.7564
Epoch 22/30
 - 5s - lo

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=450, units=400)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=120)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  


Epoch 1/15
 - 6s - loss: 2.0835 - acc: 0.4655
Epoch 2/15
 - 6s - loss: 1.2269 - acc: 0.6473
Epoch 3/15
 - 6s - loss: 1.0595 - acc: 0.6771
Epoch 4/15
 - 6s - loss: 0.9838 - acc: 0.6895
Epoch 5/15
 - 7s - loss: 0.9383 - acc: 0.6993
Epoch 6/15
 - 6s - loss: 0.9022 - acc: 0.7063
Epoch 7/15
 - 6s - loss: 0.8785 - acc: 0.7138
Epoch 8/15
 - 6s - loss: 0.8557 - acc: 0.7177
Epoch 9/15
 - 6s - loss: 0.8319 - acc: 0.7246
Epoch 10/15
 - 6s - loss: 0.8151 - acc: 0.7283
Epoch 11/15
 - 6s - loss: 0.8020 - acc: 0.7316
Epoch 12/15
 - 6s - loss: 0.7840 - acc: 0.7367
Epoch 13/15
 - 6s - loss: 0.7734 - acc: 0.7375
Epoch 14/15
 - 6s - loss: 0.7599 - acc: 0.7429
Epoch 15/15
 - 5s - loss: 0.7472 - acc: 0.7465


C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=450, units=500)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=200)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=200)`
  import sys
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/15
 - 9s - loss: 1.9203 - acc: 0.4920
Epoch 2/15
 - 9s - loss: 1.2104 - acc: 0.6450
Epoch 3/15
 - 9s - loss: 1.0710 - acc: 0.6709
Epoch 4/15
 - 9s - loss: 0.9911 - acc: 0.6885
Epoch 5/15
 - 9s - loss: 0.9481 - acc: 0.6969
Epoch 6/15
 - 9s - loss: 0.9088 - acc: 0.7042
Epoch 7/15
 - 9s - loss: 0.8768 - acc: 0.7125
Epoch 8/15
 - 9s - loss: 0.8456 - acc: 0.7218
Epoch 9/15
 - 9s - loss: 0.8285 - acc: 0.7260
Epoch 10/15
 - 10s - loss: 0.8010 - acc: 0.7324
Epoch 11/15
 - 9s - loss: 0.7818 - acc: 0.7367
Epoch 12/15
 - 9s - loss: 0.7640 - acc: 0.7423
Epoch 13/15
 - 9s - loss: 0.7477 - acc: 0.7475
Epoch 14/15
 - 9s - loss: 0.7309 - acc: 0.7504
Epoch 15/15
 - 9s - loss: 0.7187 - acc: 0.7553


C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=256)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  import sys
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=64)`
  if __name__ == '__main__':
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  if sys.path[0] == '':


Epoch 1/15
 - 10s - loss: 2.2688 - acc: 0.3902
Epoch 2/15
 - 10s - loss: 1.4692 - acc: 0.5962
Epoch 3/15
 - 10s - loss: 1.2656 - acc: 0.6399
Epoch 4/15
 - 9s - loss: 1.1617 - acc: 0.6634
Epoch 5/15
 - 10s - loss: 1.0952 - acc: 0.6795
Epoch 6/15
 - 10s - loss: 1.0490 - acc: 0.6875
Epoch 7/15
 - 9s - loss: 1.0085 - acc: 0.6957
Epoch 8/15
 - 9s - loss: 0.9767 - acc: 0.7046
Epoch 9/15
 - 9s - loss: 0.9523 - acc: 0.7101
Epoch 10/15
 - 10s - loss: 0.9187 - acc: 0.7169
Epoch 11/15
 - 9s - loss: 0.9005 - acc: 0.7227
Epoch 12/15
 - 9s - loss: 0.8800 - acc: 0.7287
Epoch 13/15
 - 9s - loss: 0.8561 - acc: 0.7334
Epoch 14/15
 - 10s - loss: 0.8374 - acc: 0.7364
Epoch 15/15
 - 11s - loss: 0.8239 - acc: 0.7394


C:\Users\Xiuquan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


'0:09:37.747838'

In [33]:
y_pred = sclf.predict(X_test)

In [34]:
from sklearn.metrics import log_loss, accuracy_score, classification_report

In [35]:
accuracy_score(y_test, y_pred)

0.7328483632258874

In [36]:
preds = sclf.predict_proba(X_test)
log_loss(y_test, preds)

0.9709868839121051

In [55]:
lr = LogisticRegression(C=0.5, solver='lbfgs', multi_class='multinomial', random_state=42)

In [56]:
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3, clf4], use_probas=True, average_probas=False, meta_classifier=lr)

In [57]:
start = datetime.now()
hist_sclf = sclf.fit(X_train, y_train)
end = datetime.now()
str(end-start)

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=450, units=512)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  


Epoch 1/30
 - 6s - loss: 2.6061 - acc: 0.4178
Epoch 2/30
 - 5s - loss: 1.7224 - acc: 0.5924
Epoch 3/30
 - 6s - loss: 1.3565 - acc: 0.6386
Epoch 4/30
 - 6s - loss: 1.1770 - acc: 0.6636
Epoch 5/30
 - 6s - loss: 1.0696 - acc: 0.6817
Epoch 6/30
 - 6s - loss: 1.0070 - acc: 0.6924
Epoch 7/30
 - 5s - loss: 0.9607 - acc: 0.6997
Epoch 8/30
 - 5s - loss: 0.9216 - acc: 0.7070
Epoch 9/30
 - 5s - loss: 0.8937 - acc: 0.7130
Epoch 10/30
 - 5s - loss: 0.8673 - acc: 0.7180
Epoch 11/30
 - 5s - loss: 0.8441 - acc: 0.7236
Epoch 12/30
 - 5s - loss: 0.8240 - acc: 0.7269
Epoch 13/30
 - 5s - loss: 0.8069 - acc: 0.7326
Epoch 14/30
 - 5s - loss: 0.7923 - acc: 0.7346
Epoch 15/30
 - 6s - loss: 0.7789 - acc: 0.7387
Epoch 16/30
 - 5s - loss: 0.7650 - acc: 0.7419
Epoch 17/30
 - 5s - loss: 0.7528 - acc: 0.7446
Epoch 18/30
 - 5s - loss: 0.7422 - acc: 0.7503
Epoch 19/30
 - 5s - loss: 0.7324 - acc: 0.7518
Epoch 20/30
 - 6s - loss: 0.7203 - acc: 0.7532
Epoch 21/30
 - 7s - loss: 0.7130 - acc: 0.7568
Epoch 22/30
 - 6s - lo

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=450, units=400)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=120)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  


Epoch 1/15
 - 6s - loss: 2.0383 - acc: 0.4847
Epoch 2/15
 - 5s - loss: 1.2286 - acc: 0.6461
Epoch 3/15
 - 6s - loss: 1.0606 - acc: 0.6767
Epoch 4/15
 - 5s - loss: 0.9848 - acc: 0.6917
Epoch 5/15
 - 5s - loss: 0.9407 - acc: 0.6999
Epoch 6/15
 - 6s - loss: 0.9029 - acc: 0.7078
Epoch 7/15
 - 5s - loss: 0.8778 - acc: 0.7113
Epoch 8/15
 - 5s - loss: 0.8518 - acc: 0.7189
Epoch 9/15
 - 6s - loss: 0.8324 - acc: 0.7245
Epoch 10/15
 - 5s - loss: 0.8143 - acc: 0.7287
Epoch 11/15
 - 5s - loss: 0.7992 - acc: 0.7328
Epoch 12/15
 - 6s - loss: 0.7850 - acc: 0.7359
Epoch 13/15
 - 6s - loss: 0.7728 - acc: 0.7390
Epoch 14/15
 - 5s - loss: 0.7593 - acc: 0.7413
Epoch 15/15
 - 6s - loss: 0.7493 - acc: 0.7438


C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=450, units=500)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=200)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=200)`
  import sys
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/15
 - 8s - loss: 1.9304 - acc: 0.4873
Epoch 2/15
 - 8s - loss: 1.2157 - acc: 0.6462
Epoch 3/15
 - 7s - loss: 1.0635 - acc: 0.6733
Epoch 4/15
 - 8s - loss: 0.9944 - acc: 0.6866
Epoch 5/15
 - 7s - loss: 0.9399 - acc: 0.6998
Epoch 6/15
 - 8s - loss: 0.9085 - acc: 0.7057
Epoch 7/15
 - 7s - loss: 0.8756 - acc: 0.7149
Epoch 8/15
 - 8s - loss: 0.8465 - acc: 0.7208
Epoch 9/15
 - 7s - loss: 0.8215 - acc: 0.7272
Epoch 10/15
 - 8s - loss: 0.8018 - acc: 0.7330
Epoch 11/15
 - 7s - loss: 0.7813 - acc: 0.7380
Epoch 12/15
 - 8s - loss: 0.7664 - acc: 0.7417
Epoch 13/15
 - 8s - loss: 0.7481 - acc: 0.7484
Epoch 14/15
 - 8s - loss: 0.7312 - acc: 0.7511
Epoch 15/15
 - 8s - loss: 0.7163 - acc: 0.7553


C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=256)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  import sys
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=64)`
  if __name__ == '__main__':
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  if sys.path[0] == '':


Epoch 1/15
 - 10s - loss: 2.2625 - acc: 0.3984
Epoch 2/15
 - 9s - loss: 1.4504 - acc: 0.6040
Epoch 3/15
 - 10s - loss: 1.2463 - acc: 0.6476
Epoch 4/15
 - 9s - loss: 1.1526 - acc: 0.6670
Epoch 5/15
 - 8s - loss: 1.0882 - acc: 0.6795
Epoch 6/15
 - 8s - loss: 1.0380 - acc: 0.6906
Epoch 7/15
 - 8s - loss: 1.0024 - acc: 0.6990
Epoch 8/15
 - 8s - loss: 0.9692 - acc: 0.7061
Epoch 9/15
 - 9s - loss: 0.9397 - acc: 0.7114
Epoch 10/15
 - 10s - loss: 0.9137 - acc: 0.7195
Epoch 11/15
 - 9s - loss: 0.8925 - acc: 0.7234
Epoch 12/15
 - 9s - loss: 0.8731 - acc: 0.7287
Epoch 13/15
 - 10s - loss: 0.8561 - acc: 0.7310
Epoch 14/15
 - 10s - loss: 0.8372 - acc: 0.7369
Epoch 15/15
 - 12s - loss: 0.8151 - acc: 0.7420


C:\Users\Xiuquan\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


'0:09:15.781186'

In [58]:
y_pred = sclf.predict(X_test)

In [59]:
accuracy_score(y_test, y_pred)

0.7308415903674903

In [60]:
y_proba = sclf.predict_proba(X_test)
log_loss(y_test, y_proba)

0.9807001358130999

In [101]:
def create_model_1hl(nodes_l1=200, dropout_l1=0.3):
    model = Sequential()
    model.add(Dense(output_dim=nodes_l1, activation="relu", input_dim=38*4))
    model.add(Dropout(dropout_l1))
            
    model.add(Dense(output_dim=len(trip_types), activation="softmax"))
    
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    
    return model

In [102]:
clf1 = KerasClassifier(build_fn=create_model_1hl, epochs=30, batch_size=1000, verbose=2)

In [103]:
sclf = StackingClassifier(classifiers=[clf2, clf2, clf3, clf4], use_probas=True, average_probas=False, meta_classifier=clf1)

In [89]:
start = datetime.now()
hist_sclf_nn = sclf.fit(X_train, y_train)
end = datetime.now()
str(end-start)

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=450, units=400)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=120)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  


Epoch 1/15
 - 96s - loss: 2.0128 - acc: 0.4838
Epoch 2/15
 - 7s - loss: 1.2140 - acc: 0.6481
Epoch 3/15
 - 7s - loss: 1.0534 - acc: 0.6774
Epoch 4/15
 - 7s - loss: 0.9832 - acc: 0.6892
Epoch 5/15
 - 6s - loss: 0.9335 - acc: 0.6991
Epoch 6/15
 - 7s - loss: 0.8998 - acc: 0.7085
Epoch 7/15
 - 7s - loss: 0.8715 - acc: 0.7148
Epoch 8/15
 - 7s - loss: 0.8501 - acc: 0.7180
Epoch 9/15
 - 7s - loss: 0.8298 - acc: 0.7230
Epoch 10/15
 - 6s - loss: 0.8123 - acc: 0.7292
Epoch 11/15
 - 7s - loss: 0.7971 - acc: 0.7317
Epoch 12/15
 - 6s - loss: 0.7797 - acc: 0.7368
Epoch 13/15
 - 7s - loss: 0.7672 - acc: 0.7410
Epoch 14/15
 - 7s - loss: 0.7572 - acc: 0.7428
Epoch 15/15
 - 6s - loss: 0.7438 - acc: 0.7482
Epoch 1/15
 - 13s - loss: 2.0864 - acc: 0.4770
Epoch 2/15
 - 7s - loss: 1.2345 - acc: 0.6454
Epoch 3/15
 - 7s - loss: 1.0637 - acc: 0.6755
Epoch 4/15
 - 7s - loss: 0.9855 - acc: 0.6896
Epoch 5/15
 - 7s - loss: 0.9387 - acc: 0.7016
Epoch 6/15
 - 7s - loss: 0.9023 - acc: 0.7092
Epoch 7/15
 - 7s - loss: 0

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=450, units=500)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=200)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=200)`
  import sys
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/15
 - 18s - loss: 1.9481 - acc: 0.4777
Epoch 2/15
 - 10s - loss: 1.2219 - acc: 0.6419
Epoch 3/15
 - 10s - loss: 1.0719 - acc: 0.6695
Epoch 4/15
 - 10s - loss: 0.9994 - acc: 0.6837
Epoch 5/15
 - 9s - loss: 0.9462 - acc: 0.6980
Epoch 6/15
 - 9s - loss: 0.9159 - acc: 0.7040
Epoch 7/15
 - 9s - loss: 0.8825 - acc: 0.7121
Epoch 8/15
 - 9s - loss: 0.8523 - acc: 0.7175
Epoch 9/15
 - 10s - loss: 0.8271 - acc: 0.7258
Epoch 10/15
 - 9s - loss: 0.8066 - acc: 0.7290
Epoch 11/15
 - 9s - loss: 0.7884 - acc: 0.7341
Epoch 12/15
 - 9s - loss: 0.7690 - acc: 0.7401
Epoch 13/15
 - 9s - loss: 0.7530 - acc: 0.7445
Epoch 14/15
 - 9s - loss: 0.7328 - acc: 0.7496
Epoch 15/15
 - 9s - loss: 0.7225 - acc: 0.7534


C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=114, units=160)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  


Epoch 1/20
 - 6s - loss: 2.2669 - acc: 0.6966
Epoch 2/20
 - 2s - loss: 0.9971 - acc: 0.7787
Epoch 3/20
 - 2s - loss: 0.8083 - acc: 0.7906
Epoch 4/20
 - 2s - loss: 0.7672 - acc: 0.7924
Epoch 5/20
 - 2s - loss: 0.7515 - acc: 0.7931
Epoch 6/20
 - 2s - loss: 0.7407 - acc: 0.7934
Epoch 7/20
 - 2s - loss: 0.7308 - acc: 0.7945
Epoch 8/20
 - 2s - loss: 0.7249 - acc: 0.7945
Epoch 9/20
 - 2s - loss: 0.7182 - acc: 0.7957
Epoch 10/20
 - 2s - loss: 0.7103 - acc: 0.7957
Epoch 11/20
 - 2s - loss: 0.7046 - acc: 0.7964
Epoch 12/20
 - 2s - loss: 0.6989 - acc: 0.7963
Epoch 13/20
 - 2s - loss: 0.6923 - acc: 0.7961
Epoch 14/20
 - 2s - loss: 0.6887 - acc: 0.7963
Epoch 15/20
 - 2s - loss: 0.6831 - acc: 0.7969
Epoch 16/20
 - 2s - loss: 0.6773 - acc: 0.7980
Epoch 17/20
 - 2s - loss: 0.6724 - acc: 0.7980
Epoch 18/20
 - 2s - loss: 0.6681 - acc: 0.7985
Epoch 19/20
 - 2s - loss: 0.6652 - acc: 0.7984
Epoch 20/20
 - 2s - loss: 0.6595 - acc: 0.7982


'0:09:03.000507'

In [90]:
y_pred = sclf.predict(X_test)

In [91]:
accuracy_score(y_test, y_pred)

0.7324302855470547

In [92]:
y_proba = sclf.predict_proba(X_test)
log_loss(y_test, y_proba)

0.9196499003079076

In [95]:
from xgboost import XGBClassifier

In [104]:
def create_model_1hl(nodes_l1=512, dropout_l1=0.3):
    model = Sequential()
    model.add(Dense(output_dim=nodes_l1, activation="relu", input_dim=len(X.columns)))
    model.add(Dropout(dropout_l1))
            
    model.add(Dense(output_dim=len(trip_types), activation="softmax"))
    
    model.compile(loss="categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])
    
    return model

In [105]:
clf1 = KerasClassifier(build_fn=create_model_1hl, epochs=30, batch_size=3000, verbose=2)

In [106]:
xgbc = XGBClassifier(n_estimators=100, min_samples_split=5, min_samples_leaf=2, max_features='sqrt', 
                     max_depth=50, bootstrap=False, random_state=42)

In [107]:
sclf = StackingClassifier(classifiers=[clf1, clf2, clf3, clf4], use_probas=True, average_probas=False, meta_classifier=xgbc)

In [108]:
start = datetime.now()
hist_sclf_nn_xgb = sclf.fit(X_train, y_train)
end = datetime.now()
str(end-start)

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=450, units=512)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  


Epoch 1/30
 - 13s - loss: 2.6139 - acc: 0.4162
Epoch 2/30
 - 6s - loss: 1.7425 - acc: 0.5929
Epoch 3/30
 - 7s - loss: 1.3743 - acc: 0.6359
Epoch 4/30
 - 6s - loss: 1.1867 - acc: 0.6612
Epoch 5/30
 - 6s - loss: 1.0832 - acc: 0.6779
Epoch 6/30
 - 5s - loss: 1.0154 - acc: 0.6900
Epoch 7/30
 - 6s - loss: 0.9694 - acc: 0.6977
Epoch 8/30
 - 6s - loss: 0.9332 - acc: 0.7046
Epoch 9/30
 - 5s - loss: 0.8996 - acc: 0.7107
Epoch 10/30
 - 6s - loss: 0.8752 - acc: 0.7162
Epoch 11/30
 - 6s - loss: 0.8510 - acc: 0.7208
Epoch 12/30
 - 5s - loss: 0.8309 - acc: 0.7271
Epoch 13/30
 - 6s - loss: 0.8147 - acc: 0.7300
Epoch 14/30
 - 6s - loss: 0.7994 - acc: 0.7333
Epoch 15/30
 - 5s - loss: 0.7853 - acc: 0.7371
Epoch 16/30
 - 6s - loss: 0.7717 - acc: 0.7397
Epoch 17/30
 - 5s - loss: 0.7608 - acc: 0.7424
Epoch 18/30
 - 6s - loss: 0.7473 - acc: 0.7463
Epoch 19/30
 - 6s - loss: 0.7374 - acc: 0.7492
Epoch 20/30
 - 5s - loss: 0.7278 - acc: 0.7536
Epoch 21/30
 - 5s - loss: 0.7188 - acc: 0.7547
Epoch 22/30
 - 6s - l

C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=450, units=400)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=120)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  


Epoch 1/15
 - 33s - loss: 2.0614 - acc: 0.4814
Epoch 2/15
 - 6s - loss: 1.2396 - acc: 0.6439
Epoch 3/15
 - 7s - loss: 1.0672 - acc: 0.6755
Epoch 4/15
 - 7s - loss: 0.9867 - acc: 0.6908
Epoch 5/15
 - 6s - loss: 0.9393 - acc: 0.6978
Epoch 6/15
 - 7s - loss: 0.9075 - acc: 0.7076
Epoch 7/15
 - 7s - loss: 0.8757 - acc: 0.7116
Epoch 8/15
 - 7s - loss: 0.8533 - acc: 0.7182
Epoch 9/15
 - 7s - loss: 0.8318 - acc: 0.7235
Epoch 10/15
 - 7s - loss: 0.8167 - acc: 0.7258
Epoch 11/15
 - 7s - loss: 0.7993 - acc: 0.7313
Epoch 12/15
 - 7s - loss: 0.7824 - acc: 0.7383
Epoch 13/15
 - 7s - loss: 0.7719 - acc: 0.7386
Epoch 14/15
 - 6s - loss: 0.7616 - acc: 0.7441
Epoch 15/15
 - 7s - loss: 0.7461 - acc: 0.7473


C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", input_dim=450, units=500)`
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=200)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=200)`
  import sys
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  # Remove the CWD from sys.path while we load stuff.


Epoch 1/15
 - 14s - loss: 1.9375 - acc: 0.4820
Epoch 2/15
 - 9s - loss: 1.2149 - acc: 0.6446
Epoch 3/15
 - 9s - loss: 1.0666 - acc: 0.6721
Epoch 4/15
 - 9s - loss: 0.9966 - acc: 0.6852
Epoch 5/15
 - 9s - loss: 0.9457 - acc: 0.6972
Epoch 6/15
 - 9s - loss: 0.9024 - acc: 0.7085
Epoch 7/15
 - 9s - loss: 0.8722 - acc: 0.7136
Epoch 8/15
 - 10s - loss: 0.8446 - acc: 0.7213
Epoch 9/15
 - 14s - loss: 0.8239 - acc: 0.7259
Epoch 10/15
 - 14s - loss: 0.8002 - acc: 0.7325
Epoch 11/15
 - 15s - loss: 0.7771 - acc: 0.7385
Epoch 12/15
 - 15s - loss: 0.7590 - acc: 0.7417
Epoch 13/15
 - 14s - loss: 0.7443 - acc: 0.7475
Epoch 14/15
 - 14s - loss: 0.7269 - acc: 0.7540
Epoch 15/15
 - 14s - loss: 0.7082 - acc: 0.7583


C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=256)`
  """
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
  import sys
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=64)`
  if __name__ == '__main__':
C:\Users\Xiuquan\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=38)`
  if sys.path[0] == '':


Epoch 1/15
 - 21s - loss: 2.2342 - acc: 0.4000
Epoch 2/15
 - 15s - loss: 1.4662 - acc: 0.5994
Epoch 3/15
 - 14s - loss: 1.2598 - acc: 0.6446
Epoch 4/15
 - 15s - loss: 1.1561 - acc: 0.6649
Epoch 5/15
 - 15s - loss: 1.0865 - acc: 0.6792
Epoch 6/15
 - 15s - loss: 1.0396 - acc: 0.6901
Epoch 7/15
 - 15s - loss: 1.0007 - acc: 0.6980
Epoch 8/15
 - 15s - loss: 0.9737 - acc: 0.7031
Epoch 9/15
 - 15s - loss: 0.9452 - acc: 0.7120
Epoch 10/15
 - 14s - loss: 0.9209 - acc: 0.7169
Epoch 11/15
 - 13s - loss: 0.8953 - acc: 0.7232
Epoch 12/15
 - 13s - loss: 0.8728 - acc: 0.7286
Epoch 13/15
 - 13s - loss: 0.8552 - acc: 0.7318
Epoch 14/15
 - 13s - loss: 0.8400 - acc: 0.7351
Epoch 15/15
 - 14s - loss: 0.8177 - acc: 0.7409


'20:33:41.892828'

In [109]:
y_pred = sclf.predict(X_test)

In [110]:
accuracy_score(y_test, y_pred)

0.7261591203645638

In [111]:
y_proba = sclf.predict_proba(X_test)
log_loss(y_test, y_proba)

1.1094501367827299

In [113]:
# save the model to disk
filename = 'sclf_nn_with_xgb.sav'
pickle.dump(hist_sclf_nn_xgb, open(filename, 'wb'))

In [114]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)

In [115]:
result

0.7261591203645638

In [117]:
log_loss(y_test, loaded_model.predict_proba(X_test))

1.1094501367827299